In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

%config Completer.use_jedi = False

In [ ]:
!wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip
!unzip ./complaints.csv.zip


In [ ]:
import torch
import transformers
#!pip install Tez
#import tez
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, texts, targets, max_len = 64):
        self.texts = texts
        self.targets = targets
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
                        "bert-base-uncased", do_lower_case = True
        )
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens = True,
                max_length = self.max_len,
                padding = "max_length",
                truncation = True
        )
        resp = {
            "ids" : torch.tensor(inputs["input_ids"], dtype=torch.long),
            "mask" : torch.tensor(inputs["attention_mask"], dtype=torch.long),
            "token_type_ids" : torch.tensor(inputs["token_type_ids"], dtype=torch.long),
            "targets" : torch.tensor(self.targets["idx"], dtype = torch.float)
        }
        return resp
    
    
class TextModel(tez.Model):
    def __init__(self, num_classes, num_train_steps):
        super().__init__()
        self.bert = transformers.BertModel.from_pretrained(
                    "bert-base-uncased", return_dict = False
        )
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, num_classes)
        self.num_train_steps = num_train_steps
        self.optim_ = self.fetch_optimizer()
        
    def fetch_optimizer(self):
        opt = AdamW(self.parameters(), lr = 1e-4)
        
    def fetch_scheduler(self):
        sch = get_linear_schedule_with_warmup(
            AdamW(self.parameters(), lr = 1e-4), num_warmup_steps = 0,num_training_steps = self.num_train_steps
        )
        return sch
    
    
    def loss(self, outputs, targets):
        return nn.BCEWithLogitsLoss()(outputs, targets.view(-1,1))
    
    def monitor_metrics(self, outputs, targets):
        outputs = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5
        targets = targets.cpu().detach().numpy()
        return {
            "accuracy" : metrics.accruacy_score(targets, outputs)
        }
    
    def forward(self, ids, mark, token_type_ids, targets = None):
        _, x = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
        x = self.bert_drop(x)
        x = self.out(x)
        if(targets is not None):
            loss = self.loss(outputs , targets)
            metrics = self.monitor_metrics(x , targets)
            return x, loss, met
        return x, 0, {}
    

    def train_model(out1,out2):
        
        
        train_dataset = BERTDataset(out1.Title.values, out1.label.values)
        valid_dataset = BERTDataset(out2.Title.values, out2.label.values)
        
        n_train_steps = int(len(out1) / 32 * 10)
        
        model = TextModel(num_classes = 1, num_train_steps = n_train_steps, )
        
        ess = tez.callbacks.EarlyStopping(monitor = "valid_loss", patience = 3, model_path = "model.bin")
    
        model.fit(
                train_dataset,
                valid_dataset = valid_dataset,
                device = "cuda",
                epochs = 10,
                train_bs = 32,
                callbacks = [ess]
        )
    
    

    
    

In [ ]:
#!wget https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/title_conference.csv
!wget https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/title_conference.csv

In [ ]:
df = pd.read_csv('../input/200000-jeopardy-questions/JEOPARDY_CSV.csv').fillna("none")
#df['Conference'].value_counts()
# possible_label = df[" Value"].unique()
# label_dict = {}
# for index, possible_label in enumerate(possible_label):
#     label_dict[possible_label] = index

# df['label'] = df[" Value"].replace(label_dict)
# #df.head()

In [ ]:
df.head
df.isnull().sum()

In [ ]:
import seaborn as sns
sns.countplot(x=' Round',data=df)

In [ ]:
df.drop(df[df[' Value'] == 'None'].index,inplace=True)

In [ ]:
df['ValueNum'] = df[' Value'].apply(
    lambda value: int(value.replace('$', '').replace(',','')))

In [ ]:
df['ValueNum'].unique()
df['ValueNum'].nunique()
def binning(value):
    if value < 1000:
        return np.round(value, -2)
    elif value < 10000:
        return np.round(value, -3)
    else:
        return np.round(value, -4)

df['ValueBins'] = df['ValueNum'].apply(binning)

In [ ]:
import re
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
df['with_out_tags']=df[' Question'].apply(lambda cw : remove_tags(cw))

In [ ]:
new_df = df[["with_out_tags","ValueBins"]].copy()

In [ ]:
new_df['ValueBins'] = new_df['ValueBins'].astype('category')
# Assigning numerical values and storing in another column
new_df['labels'] = new_df['ValueBins'].cat.codes

In [ ]:
new_df.labels.unique()

In [ ]:
final_df = new_df[["with_out_tags","labels"]].copy()
final_df

In [ ]:
new_Df['ValueBins'].value_counts()
possible_label = new_Df["ValueBins"].unique()
label_dict = {}
for index, possible_label in enumerate(possible_label):
    label_dict[possible_label] = index

new_Df['label'] = new_Df["ValueBins"].replace(label_dict)
new_Df.head()

In [ ]:
new_Df["label"].unique()
final_df = new_Df[["with_out_tags", "label"]]

In [ ]:
df.columns

In [ ]:
from sklearn.model_selection import KFold
#new_Df = df[['with_out_tags','ValueBins']]
#added some parameters
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
result = next(kf.split(final_df), None)




train = final_df.iloc[result[0]]
test =  final_df.iloc[result[1]]

In [ ]:
train.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True) 


In [ ]:
fin_df = final_df.sample(n = 50000)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(fin_df, test_size=0.2)

In [ ]:
len(train)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df.with_out_tags.values, final_df.labels.values, test_size=0.33,random_state=1)

In [ ]:
train, test = train_test_split(final_df, test_size=0.2, random_state=2019)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_type = "Out"
model_name = "roberta"

train.columns = ["text", "labels"]
train_args = {
    'output_dir': f'{model_type}-{model_name}-outputs',

    'max_seq_length': 128,
    'num_train_epochs': 5,
    'train_batch_size': 16,
    'eval_batch_size': 32,
    'gradient_accumulation_steps': 1,
    'learning_rate': 5e-5,
    'save_steps': -1,

    'wandb_project': 'Wysa',
    'wandb_kwargs': {'name': f'{model_type}-{model_name}'},
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 1000,
    'reprocess_input_data': True,
    "save_model_every_epoch": False,
    'overwrite_output_dir': True,
    'no_cache': True,
    'no_save' : True,

    'use_early_stopping': True,
    'early_stopping_patience': 3,
    'manual_seed': 4,
}




test.columns = ["text", "labels"]


# Optional model configuration
#model_args = ClassificationArgs(num_train_epochs=8)

# Create a ClassificationModel
model = ClassificationModel(
'roberta', 'roberta-base',
    num_labels=20,
    #args=model_args,
    args=train_args
) 

#model = ClassificationModel(model_type, model_name, num_labels=4, args=train_args)
# Train the model
model.train_model(train,eval_df=test)

# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(test)

# Make predictions with the model
#predictions, raw_outputs = model.predict(["Sam was a Wizard"])


In [ ]:
!ls

In [ ]:
du - h ./final-roberta-outputs

In [ ]:
du -h final-roberta-outputs/

In [ ]:
ls -ltr

In [ ]:
model_best = ClassificationModel('roberta', 'outputs/best_model/', num_labels=5)

In [ ]:
ls

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
#result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

In [ ]:
result

In [ ]:
!pip install simpletransformers

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
bow = CountVectorizer(stop_words='english', max_features=2000)
bow.fit(train['text'])

In [ ]:
X_train = bow.transform(train["text"])
X_test = bow.transform(test["text"])

y_train = train.labels
y_test = test.labels

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
lr = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=200)
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

print(classification_report(y_test, y_pred))